# Traffic&nbsp;Sign&nbsp;Classification with a Deep Convolutional&nbsp;Neural&nbsp;Network

by Karl J. Obermeyer

## Overview

In this project, I implemented a deep CNN (Convolutional Neural Network) in TensorFlow to classify traffic sign images. The code is in the Jupyter notebook `traffic_sign_classification.ipynb` and custom python module `image_recognition_tools.py`. The notebook can be run according to the instructions in `README.md`. Image data used for training, validation, and testing were from the [GTSRB (German Traffic Sign Recognition Benchmark)](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset). The network architecture is based loosely on [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf). Initially I experimented with a CNN that was essentially the same as LeNet-5, but with only the number of outputs changed from 10 to 43.  This baseline CNN achieved a mere 89.0% classification accuracy on the GTSRB validation set.  To improve performance, I took the following steps:

  1. Increased the depth of convolutional filters,  
  2. Added more nodes to fully connected layers, and  
  3. Added dropout regularization. 

The final trained CNN achieved 96.5% classification accuracy on the GTSRB test set, 95.8% accuracy on a balanced version of the GTSRB test set, and 80% classification accuracy on five independently obtained images. Examining the top few softmax probabilities for the independently obtained images showed that the CNN did have some uncertainty in distinguishing between signs that were geometrically similar and look similar, in most cases, even to humans in grayscale.  Future work should explore different network topologies and inputs richer than just grayscale, e.g., full RGB together with grayscale and possibly edges detected in preprocessing.

## Data Summary, Exploration, and Balancing

The GTSRB datasets, which consisted of integer-labelled RGB images (each 32x32x3), had the following properties.

Set | Count
----:|:------  
training examples | 34799  
validation examples | 4410  
testing examples | 12630  
unique classes | 43  

To get an idea of what type of sign each class represents, here are some summary images I synthesized from all images in the datasets (training, validation and testing images combined). Summary images can be created in a variety of ways, e.g., by choosing one example, taking the pixelwise mean, or the pixelwise median.  I used the pixelwise median.

<img src="fig/class_representative_images.png"
 alt="class representative images" style="width: 600px; margin: 0 auto"/>

Next, we look at how balanced the data are by making histograms of the labels in each of the different datasets.

<img src="fig/class_histograms.png"
 alt="class histograms" style="width: 700px; margin: 0 auto"/>

Each class seems to have fairly equal representation across data subsets (train, validate, test), but some classes have greater representation within each subset. We must be wary that training with unbalanced data can bias the final classifier away from under-represented classes. Therefore, we construct a more balanced dataset.

### Data Balancing

To create more balanced datasets, we augment the data by stratified sampling together with random image perturbation. Image perturbation is implemented in the `randomly_perturb` function in `image_recognition_tools.py`. Images are perturbed by random changes in brightness, rotation, translation, and shear. This helps train the CNN to classify images captured under a greater variety of conditions. Here are some examples of perturbed versions of an image.

<img src="fig/image_perturbations_example.png"
 alt="image perturbation example" style="width: 600px; margin: 0 auto"/>

Histograms comfirm that balancing of the datasets worked.

<img src="fig/class_histograms-balanced.png"
 alt="balanced class histograms" style="width: 700px; margin: 0 auto"/>

This table shows the total numbers of examples in the balanced datasets.

Set | Count
---------|:------  
balanced training examples | 129000
balanced validation examples | 10320
balanced training + validation | 139320
balanced testing examples | 32250

## CNN Architecture

We use an architecture similar to [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf), but we have expanded the dimensions of convolution filters and fully connected layers to accomodate the greater number of classes in our dataset (43 vs 10).  Also, we include a dropout regularization.

<img src="fig/cnn_architecture-horizontal.png"
 alt="cnn architecture" style="width: 700px; margin: 0 auto"/>

Each layer of the network maps dimensions as follows.

Layer | Type      | Input | Output
------|:----------|:------|:--------
n/a   | RGB image | n/a | 32x32x3
0     | Preprocessing | 32x32x3 | 32x32x1
1     | Convolutional | 32x32x1 | 28x28x16
2     | Pooling | 28x28x16 | 14x14x16.
3     | Convolutional | 14x14x16 | 10x10x32.
4     | Pooling | 10x10x32 | 5x5x32
5     | Flatten layer | 5x5x32 | 800
6     | Fully Connected | 800 | 600
7     | Fully Connected | 600 | 500
8     | Fully Connected | 500 | 43

### Image Preprocessing: Brightness Equalization, Grayscale, and&nbsp;Normalization

Brightness equalization using the histogram technique, implemented in OpenCV, helps our CNN perform more consistantly across varying lighting conditions. This figure shows the nominal image in the upper left. The left column and right column show unequalized and equalized versions, respectively, of the color and grayscale images.

<img src="fig/histogram_brightness_equalization.png"
 alt="histogram brightness equalization example" style="width: 400px; margin: 0 auto"/>

Since shape and markings are much more important for sign classification than color, we help our CNN focus on what's most important by grayscaling images. Finally, normalizing pixel values to the range [-1.0, 1.0] helps mitigate optimization problems that can be caused by scale differences between features learned by the CNN.

## Training, Validation, and Testing

For training, I used the Adam optimization algorithm, a variant of stochastic gradient descent which uses momentum and learning rates that adapt individually on each parameter. It is known to work well for cases like ours where dropout regularization and only small batches of data are used to estimate the gradient at each iteration [2]. Network weights and biases were initialized with random truncated normal samples. Hyperparameters I fixed to the following values for all validation and testing.  
   `LEARNING_RATE = 0.001`  
   `EPOCH_CNT = 30`  
   `BATCH_SIZE = 256`  

### Validation

Validation runs were performed to determine a good `KEEP_PROBABILITY` value for dropout regularization.  Of the validated values, 0.4 was selected for final testing.

`KEEP_PROBABILITY` | train. accuracy | valid. accuracy | balanced valid. accuracy
:-------------------:|:----------------:|:----------------:|:-------------------------:
1.0 | 1.000 | 0.968 | 0.953
0.7 | 0.999 | 0.978 | 0.973
0.5 | 1.000 | 0.973 | 0.961
0.4 | 0.999 | 0.988 | 0.966
0.3 | 0.998 | 0.980 | 0.975

### Final Testing

For the final test, I trained using a combined balanced training and balanced validation set.  This means I trained on 129000 + 10320 = 139320 examples.

`KEEP_PROBABILITY` | train. accuracy | test accuracy | balanced test accuracy
:-------------------:|:----------------:|:----------------:|:---------------------:
0.4 | 0.999 | 0.965 | 0.958 

<img src="fig/accuracies_vs_epoch-final.png"
 alt="accuracies vs epoch" style="width: 500px; margin: 0 auto"/>

Since the gap between training and test accuracy is relatively small, and the accuracies increased roughly monotonically during training, I believe the model is well-fitting (i.e. not over- or underfitting).

#### Performance on Independently Obtained Images

For the 5 independently obtained Creative Commons images below, the final classifier performed perfectly. This is not surprising considering that these images were all in focus and taken in good lighting conditions from a relatively direct angle.  However, plots of the top 5 softmax probabilities reveal the small amount of uncertainty in the classifications. As one would expect, the classes assigned the highest probabilities in each case correspond mostly to signs that look similar even to humans, at least in grayscale. Images 2 (children crossing) and 4 (roundabout mandatory) had the most uncertanty.  Image 2 was confused mostly with other triangular signs with black symbols in the center.  Image 4 was confused with other round sinds with a thick border, presumably because the circular arrows mimick the inside edge of signs with thick border, e.g., speed limit signs. In a larger set of images, I imagine the classifier would become uncertain about images with noise, blurring, poor lighting, oblique viewing angle, etc.

Image 0  
  Truth class 14: Stop  
  Predicted class 14: Stop  
<img src="fig/new_image-000.top_softmax_probabilities.png"
 alt="softmax probabilities" style="width: 600px; margin: 0 auto"/>

Image 1  
  Truth class 17: No entry  
  Predicted class 17: No entry  
<img src="fig/new_image-001.top_softmax_probabilities.png"
 alt="softmax probabilities" style="width: 600px; margin: 0 auto"/>

Image 2  
  Truth class 28: Children crossing  
  Predicted class 11: Right-of-way at the next intersection  
<img src="fig/new_image-002.top_softmax_probabilities.png"
 alt="softmax probabilities" style="width: 600px; margin: 0 auto"/>

Image 3  
  Truth class 39: Keep left  
  Predicted class 39: Keep left  
<img src="fig/new_image-003.top_softmax_probabilities.png"
 alt="softmax probabilities" style="width: 600px; margin: 0 auto"/>

Image 4  
  Truth class 40: Roundabout mandatory  
  Predicted class 40: Roundabout mandatory  
<img src="fig/new_image-004.top_softmax_probabilities.png"
 alt="softmax probabilities" style="width: 600px; margin: 0 auto"/>

## Conclusions

In this project, I showed a successful TensorFlow implementation of a deep CNN (Convolutional Neural Network) for classifying traffic sign images based on the GTSRB (German Traffic Sign Recognition Benchmark).  The architecture was chosen by scaling up LeNet-5 [1] and adding dropout regularization. This achieved 96.5% classification accuracy on the GTSRB test set, 95.8%  accuracy on a balanced version of the GTSRB test set, and 80% classification accuracy on five independently obtained images. Examining the top few softmax probabilities for the independently obtained images showed that the CNN did have some uncertainty in distinguishing between signs that were geometrically similar, and which in most casses look similar even to humans in grayscale.

### Limitations, Weaknesses, and Future Work

While 96.5% accuracy is a significant improvement over the 89% accuracy of the baseline CNN, there is still work to be done if the classifier is to be used in  real-time safety critical scenarios. For the next version of the CNN, I would consider the following modifications.

 * Change the input images to include all RGB colors, grayscale, and possibly even precomputed Canny edges.  

 * Try topologically different architectures, e.g., feed-forward from the first convolution to the first fully connected layer.

 * Gather more data and/or use more sophisticated data augmentation.

## References

[0] J. Stallkamp and M. Schlipsing and J. Salmen and C. Igel, [The German Traffic Sign Recognition Benchmark: A multi-class classification competition](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset), IEEE International Joint Conference on Neural Networks, 2011, p. 1453-1460.

[1] Y. LeCun and L. Bottou and Y. Bengio and P. Haffner, [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf), Proceedings of the IEEE, 86(11):2278-2324, November 1998. Note: Lenet-5.

[2] G. E Hinton and N. Srivastava and A. Krizhevsky and I. Sutskever and R. R. Salakhutdinov, "Improving neural networks by preventing co-adaptation of feature detectors", ArXiv preprint, [https://arxiv.org/abs/1207.0580](https://arxiv.org/abs/1207.0580). Note: Dropout.

[3] P. Sermanet and Y. LeCun, [Traffic Sign Recognition with Multi-Scale Convolutional Networks](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf), 2011.